Todos os scripts usam a sintaxe T-SQL.

1. Crie uma query que retorne os valores distintos de cidade existentes na tabela de clientes:

In [6]:
-- Cidades em Estados diferentes podem ter o mesmo nome
SELECT 
    DISTINCT CONCAT(UPPER(customer_city), ' / ', customer_state) AS cidade_estado
FROM dbo.olist_customers
ORDER BY cidade_estado;

(4310 rows affected)

Total execution time: 00:00:01.053

cidade_estado
ABADIA DOS DOURADOS / MG
ABADIANIA / GO
ABAETE / MG
ABAETETUBA / PA
ABAIARA / CE
ABAIRA / BA
ABARE / BA
ABATIA / PR
ABDON BATISTA / SC
ABELARDO LUZ / SC
